## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [13]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\YLIU378\Programs\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from openai import OpenAI

In [1]:
pip show langchain


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show langchain-core

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip show langchain-community

Name: langchain-community
Version: 0.3.27
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\Users\YLIU378\Programs\Python\Lib\site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip uninstall langchain langchain-core langchain-community langchain-text-splitters -y


Found existing installation: langchain 0.3.26Note: you may need to restart the kernel to use updated packages.

Uninstalling langchain-0.3.26:
  Successfully uninstalled langchain-0.3.26
Found existing installation: langchain-core 0.3.68
Uninstalling langchain-core-0.3.68:
  Successfully uninstalled langchain-core-0.3.68
Found existing installation: langchain-community 0.3.27
Uninstalling langchain-community-0.3.27:
  Successfully uninstalled langchain-community-0.3.27
Found existing installation: langchain-text-splitters 0.3.8
Uninstalling langchain-text-splitters-0.3.8:
  Successfully uninstalled langchain-text-splitters-0.3.8


In [5]:
pip install langchain langchain-core langchain-community


  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain-0.3.26-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.68-py3-none-any.whl (441 kB)
Using cached langchain_community-0.3.27-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.document_loaders.text import TextLoader

In [7]:
from langchain_community.document_loaders import DirectoryLoader

In [8]:
# imports for langchain


from langchain.text_splitter import CharacterTextSplitter

In [ ]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [11]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [14]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase
# Thank you Mark D. and Zoya H. for fixing a bug here..

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [15]:
len(documents)

31

In [16]:
documents[24]

Document(metadata={'source': 'knowledge-base\\employees\\Oliver Spencer.md', 'doc_type': 'employees'}, page_content='# HR Record\n\n# Oliver Spencer\n\n## Summary\n- **Date of Birth**: May 14, 1990  \n- **Job Title**: Backend Software Engineer  \n- **Location**: Austin, Texas  \n\n## Insurellm Career Progression\n- **March 2018**: Joined Insurellm as a Backend Developer I, focusing on API development for customer management systems.\n- **July 2019**: Promoted to Backend Developer II after successfully leading a team project to revamp the claims processing system, reducing response time by 30%.\n- **June 2021**: Transitioned to Backend Software Engineer with a broader role in architecture and system design, collaborating closely with the DevOps team.\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January 2023**: Awarded a mentorship role to guide new hires in backend technology

In [17]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [18]:
len(chunks)

123

In [19]:
chunks[6]

Document(metadata={'source': 'knowledge-base\\contracts\\Contract with Apex Reinsurance for Rellm.md', 'doc_type': 'contracts'}, page_content="1. **Technical Support**: Provider shall offer dedicated technical support to the Client via phone, email, and a ticketing system during business hours (Monday to Friday, 9 AM to 5 PM EST).\n\n2. **Training and Onboarding**: Provider will deliver comprehensive onboarding training for up to ten (10) members of the Client's staff to ensure effective use of the Rellm solution.\n\n3. **Updates and Maintenance**: Provider is responsible for providing updates to the Rellm platform to improve functionality and security, at no additional cost to the Client.\n\n4. **Escalation Protocol**: Issues that cannot be resolved at the first level of support will be escalated to the senior support team, ensuring that critical concerns are addressed promptly.\n\n---\n\n**Acceptance of Terms**: By signing below, both parties agree to the Terms, Renewal, Features, an

In [20]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: products, company, employees, contracts


In [21]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='3. **Regular Updates:** Insurellm will offer ongoing updates and enhancements to the Homellm platform, including new features and security improvements.

4. **Feedback Implementation:** Insurellm will actively solicit feedback from GreenValley Insurance to ensure Homellm continues to meet their evolving needs.

---

**Signatures:**

_________________________________  
**[Name]**  
**Title**: CEO  
**Insurellm, Inc.**

_________________________________  
**[Name]**  
**Title**: COO  
**GreenValley Insurance, LLC**  

---

This agreement represents the complete understanding of both parties regarding the use of the Homellm product and supersedes any prior agreements or communications.' metadata={'source': 'knowledge-base\\contracts\\Contract with GreenValley Insurance for Homellm.md', 'doc_type': 'contracts'}
_________
page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, availa